### Pentane-Hexane-Heptane (Ideal)

In [ ]:
%config InlineBackend.figure_format='svg'
%matplotlib inline

import chbe444umd as des

sys567 = des.make_system(components={'pen', 'hex', 'hep'},
                          Antoine={'pen': {'A': 3.989,
                                           'B': 1070.617,
                                           'C': -40.454},
                                   'hex': {'A': 4.00266,
                                           'B': 1171.53,
                                           'C': -48.784},
                                   'hep': {'A': 4.02832,
                                           'B': 1268.636,
                                           'C': -56.199}},
                          Wilson_a={'pen': {'pen': 0,
                                            'hex': 0,
                                            'hep': 0},
                                    'hex': {'pen': 0,
                                            'hex': 0,
                                            'hep': 0},
                                    'hep': {'pen': 0,
                                            'hex': 0,
                                            'hep': 0}},
                          Wilson_b={'pen': {'pen': 0,
                                            'hex': 0,
                                            'hep': 0},
                                    'hex': {'pen': 0,
                                            'hex': 0,
                                            'hep': 0},
                                    'hep': {'pen': 0,
                                            'hex': 0,
                                            'hep': 0}})

ax567 = {'pen': 0, 'hep': 1}
    # Components on axes 0 (x-axis) and 1 (y-axis) must be explicitly specified
    # in this format. Specifying the implicit component (axis 2) is optional.

vle3 = des.vle_calc(
    sys567, ax567,
    n_vectors=51, arrow_scale=51,
    contour_T_heavy=10, contour_T_medium=5, contour_T_light=1,
    plot_residue=False,
    residue_feeds={0: {'pen': 0.33, 'hex': 0.34, 'hep': 0.33},
                   1: {'pen': 0.32, 'hex': 0.28, 'hep': 0.40}})

    # n_vectors: number of e-vectors along each axis
    # arrow_scale: scale of arrows on equilibrium field
    #   (set approximately equal to n_vector)
    # tolerance to establish right triangle (set as slightly less than zero,
    #   e.g., -1e-6)
    # contour_heavy_T: heavy (and labeled) temperature contours are
    #   incremented by this temperature in °C
    # contour_medium_T: medium temperature contours are incremented by this
    #   temperature in °C
    # contour_light_T: light temperature contours are incremented by this
    #   temperature in °C
    # plot_residue: indicates whether or not residue curve will be plotted
    #   through the feed point
    # feed: coordinates of residue curve feed point

vle3.plt.savefig('penhexhep.svg', format='svg', bbox_inches='tight', pad_inches=0)

#### Equilibrium and Flash Calculations

In [ ]:
x = [0.32, 0.40, 0.28]
print(vle3.mix_props(x))

In [ ]:
x = [0.33, 0.33, 0.34]
print(vle3.mix_props(x))

In [ ]:
x = [0.32, 0.40, 0.28]
y = vle3.mix_props(x).yeq

fig, ax = vle3.duplicate(vle3.fig)
ax.plot([x[0], y[0]], [x[1], y[1]], 'k-')
ax.plot(x[0], x[1], 'bo', markersize=6)
ax.plot(y[0], y[1], 'ro', markersize=6)

fig

In [ ]:
xF = [0.33, 0.33, 0.34]

res = vle3.flashT(xF, 72)

print(res)
x = res.x
y = res.y

fig, ax = vle3.duplicate(vle3.fig)
ax.plot([x[0], xF[0], y[0]], [x[1], xF[1], y[1]], 'k-')
ax.plot(x[0], x[1], 'bo', markersize=6)
ax.plot(xF[0], xF[1], 'ko', markersize=6)
ax.plot(y[0], y[1], 'ro', markersize=6)

fig

In [ ]:
xF = [0.33, 0.33, 0.34]

res = vle3.flashVfrac(xF, 0.68)

print(res)
x = res.x
y = res.y

fig, ax = vle3.duplicate(vle3.fig)
ax.plot([x[0], xF[0], y[0]], [x[1], xF[1], y[1]], 'k-')
ax.plot(x[0], x[1], 'bo', markersize=6)
ax.plot(xF[0], xF[1], 'ko', markersize=6)
ax.plot(y[0], y[1], 'ro', markersize=6)

fig

In [ ]:
vle3.flash_bfd(res);

#### Distillation Calculations

In [ ]:
res = vle3.distill(xF=[0.33, 0.33, 0.34],
                   xD=[0.8764, 0.0064, 0.1172],
                   D=0.2640,
                   r=5,
                   q=1,
                   nstg=4,
                   fstg=None,
                   mode='minstages')

print(res)
# print(res.xLr)
# print(res.xLs)

In [ ]:
import numpy as np

xF = res.xF
xD = res.xD
xB = res.xB

xLr = res.xLr
yVr = res.yVr
xLs = res.xLs
yVs = res.yVs
nstg = res.nstg
fstg = res.fstg

fig, ax = vle3.duplicate(vle3.fig)
ax.plot([xF[0], xD[0], xB[0]], [xF[1], xD[1], xB[1]], 'ko-', markersize=10)
ax.text(xF[0], xF[1], '$x_F$ ', ha='right', va='top')
ax.text(xD[0], xD[1], '$x_D$ ', ha='right', va='top')
ax.text(xB[0], xB[1], '$x_B$ ', ha='right', va='top')

delta = 0.01

stages_r = np.size(xLr, 0)
stages_s = np.size(xLs, 0)

# R section 

ax.plot(xLr[:, 0], xLr[:, 1], 'bo-', markersize=5, linewidth=1)
ax.plot(yVr[:, 0], yVr[:, 1], 'ro-', markersize=5, linewidth=1)

for i in range(0, stages_r):
    ax.text(xLr[i, 0] + delta, xLr[i, 1] + delta, str(i + 1), color='blue', fontsize=8)
    ax.text(yVr[i, 0] + delta, yVr[i, 1] + delta, str(i + 1), color='red', fontsize=8)

# S section

ax.plot(xLs[:, 0], xLs[:, 1], 'bo--', markersize=5, linewidth=1)
ax.plot(yVs[:, 0], yVs[:, 1], 'ro--', markersize=5, linewidth=1)

for i in range(0, stages_s):
    ax.text(xLs[i, 0] + delta, xLs[i, 1] + delta, str(i + 1), color='blue', fontsize=8)
    ax.text(yVs[i, 0] + delta, yVs[i, 1] + delta, str(i + 1), color='red', fontsize=8)

fig

In [ ]:
res = vle3.distill(xF=[0.33, 0.33, 0.34],
                   xD=[0.8764, 0.0064, 0.1172],
                   D=0.2640,
                   r=0.9,
                   q=1,
                   nstg=None,
                   fstg=None,
                   mode='minreflux')

print(res)

In [ ]:
import numpy as np

xF = res.xF
xD = res.xD
xB = res.xB

xLr = res.xLr
yVr = res.yVr
xLs = res.xLs
yVs = res.yVs
nstg = res.nstg
fstg = res.fstg

fig, ax = vle3.duplicate(vle3.fig)
ax.plot([xF[0], xD[0], xB[0]], [xF[1], xD[1], xB[1]], 'ko-', markersize=10)
ax.text(xF[0], xF[1], '$x_F$ ', ha='right', va='top')
ax.text(xD[0], xD[1], '$x_D$ ', ha='right', va='top')
ax.text(xB[0], xB[1], '$x_B$ ', ha='right', va='top')

delta = 0.01

stages_r = np.size(xLr, 0)
stages_s = np.size(xLs, 0)

# R section 

ax.plot(xLr[:, 0], xLr[:, 1], 'bo-', markersize=5, linewidth=1)
ax.plot(yVr[:, 0], yVr[:, 1], 'ro-', markersize=5, linewidth=1)

for i in range(0, stages_r):
    ax.text(xLr[i, 0] + delta, xLr[i, 1] + delta, str(i + 1), color='blue', fontsize=8)
    ax.text(yVr[i, 0] + delta, yVr[i, 1] + delta, str(i + 1), color='red', fontsize=8)

# S section

ax.plot(xLs[:, 0], xLs[:, 1], 'bo--', markersize=5, linewidth=1)
ax.plot(yVs[:, 0], yVs[:, 1], 'ro--', markersize=5, linewidth=1)

for i in range(0, stages_s):
    ax.text(xLs[i, 0] + delta, xLs[i, 1] + delta, str(i + 1), color='blue', fontsize=8)
    ax.text(yVs[i, 0] + delta, yVs[i, 1] + delta, str(i + 1), color='red', fontsize=8)

fig

In [ ]:
vle3.distill_bfd(res);